In [1]:
import tensorflow as tf
import numpy as np
import os
import os.path
from random import shuffle
from PIL import Image
import sklearn.preprocessing

import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression


curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
TRAIN_DIR = 'C:\\Users\\hiremath\\Downloads\\devanagari\\Images\\vc'
VAL_DIR = 'C:\\Users\\hiremath\\Downloads\\devanagari\\Images\\t'
IMG_SIZE = 32
LR = 0.001
MODEL_NAME = 'devanagari-{}-{}.model'.format(LR,'5conv-basic')


In [3]:
def label_img(pic):
    l = pic.split('_')[-2]
    a = []
    a.insert(0,int(l))
    label_binarizer = sklearn.preprocessing.LabelBinarizer()
    label_binarizer.fit(range(46))
    b = label_binarizer.transform(a)
    return b

In [4]:
def create_train_data():
    training_data = []
    for pic in os.listdir(TRAIN_DIR):
        DIR = os.path.join(TRAIN_DIR,pic)
        for img in os.listdir(DIR):
            if img != "Thumbs - Copy.db" and img != "Thumbs.db":
                label = label_img(pic)
                path1 = os.path.join(DIR,img)
                with Image.open(path1).convert('L') as img1:
                    img1 = img1.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS)
                    training_data.append([np.array(img1),np.array(label)])
    
    
    shuffle(training_data)
    np.save('train_dev.npy',training_data)
    
    return training_data

In [5]:
def create_validation_data():
    validation_data = []
    for pic in os.listdir(VAL_DIR):
        DIR = os.path.join(VAL_DIR,pic)
        for img in os.listdir(DIR):
            if img != "Thumbs - Copy.db" and img != "Thumbs.db":
                label = label_img(pic)
                path1 = os.path.join(DIR,img)
                with Image.open(path1).convert('L') as img1:
                    img1 = img1.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS)
                    validation_data.append([np.array(img1),np.array(label)])
    
    
    shuffle(validation_data)
    np.save('validation_dev.npy',validation_data)
    return validation_data

In [6]:
train_data = create_train_data()
#train_data = np.load('train_dev.npy')

In [7]:
val_data = create_validation_data()
#val_data = np.load('validation_dev.npy')

36846
C:\Users\hiremath\Downloads\devanagari\Images\t\character_46_9\56777.png


In [8]:
tf.reset_default_graph()
convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 46, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

In [9]:
model = tflearn.DNN(convnet, tensorboard_dir='log')

In [39]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

In [10]:
train = train_data
val = val_data

In [11]:
X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1][0] for i in train]
test_x = np.array([i[0] for i in val]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1][0] for i in val]

In [18]:
model.fit({'input': X}, {'targets': Y}, n_epoch = 1,validation_set=({'input': test_x}, {'targets': test_y}), batch_size= 128, snapshot_step =1, show_metric= True, run_id=MODEL_NAME)

---------------------------------
Run id: devanagari-0.001-5conv-basic.model
Log directory: log/
---------------------------------
Training samples: 55154
Validation samples: 36846
--
Training Step: 2587  | total loss: 0.14097 | time: 37.459s
| Adam | epoch: 007 | loss: 0.14097 - acc: 0.9532 | val_loss: 0.52162 - val_acc: 0.8442 -- iter: 00128/55154
--
Training Step: 2588  | total loss: 0.13054 | time: 71.925s
| Adam | epoch: 007 | loss: 0.13054 - acc: 0.9540 | val_loss: 0.53037 - val_acc: 0.8424 -- iter: 00256/55154
--
Training Step: 2589  | total loss: 0.12120 | time: 115.864s
| Adam | epoch: 007 | loss: 0.12120 - acc: 0.9539 | val_loss: 0.52387 - val_acc: 0.8438 -- iter: 00384/55154
--
Training Step: 2590  | total loss: 0.11425 | time: 159.730s
| Adam | epoch: 007 | loss: 0.11425 - acc: 0.9515 | val_loss: 0.51390 - val_acc: 0.8460 -- iter: 00512/55154
--
Training Step: 2591  | total loss: 0.11474 | time: 207.964s
| Adam | epoch: 007 | loss: 0.11474 - acc: 0.9524 | val_loss: 0.50900 

Training Step: 2630  | total loss: 0.07068 | time: 1739.660s
| Adam | epoch: 007 | loss: 0.07068 - acc: 0.9579 | val_loss: 0.54675 - val_acc: 0.8421 -- iter: 05632/55154
--
Training Step: 2631  | total loss: 0.06655 | time: 1778.885s
| Adam | epoch: 007 | loss: 0.06655 - acc: 0.9605 | val_loss: 0.52971 - val_acc: 0.8450 -- iter: 05760/55154
--
Training Step: 2632  | total loss: 0.06457 | time: 1817.736s
| Adam | epoch: 007 | loss: 0.06457 - acc: 0.9598 | val_loss: 0.50795 - val_acc: 0.8496 -- iter: 05888/55154
--
Training Step: 2633  | total loss: 0.06606 | time: 1858.318s
| Adam | epoch: 007 | loss: 0.06606 - acc: 0.9583 | val_loss: 0.47887 - val_acc: 0.8545 -- iter: 06016/55154
--
Training Step: 2634  | total loss: 0.06830 | time: 1892.043s
| Adam | epoch: 007 | loss: 0.06830 - acc: 0.9570 | val_loss: 0.45883 - val_acc: 0.8570 -- iter: 06144/55154
--
Training Step: 2635  | total loss: 0.06682 | time: 1926.060s
| Adam | epoch: 007 | loss: 0.06682 - acc: 0.9551 | val_loss: 0.45556 - va

Training Step: 2674  | total loss: 0.07302 | time: 3341.760s
| Adam | epoch: 007 | loss: 0.07302 - acc: 0.9473 | val_loss: 0.44578 - val_acc: 0.8601 -- iter: 11264/55154
--
Training Step: 2675  | total loss: 0.06793 | time: 3383.110s
| Adam | epoch: 007 | loss: 0.06793 - acc: 0.9494 | val_loss: 0.46743 - val_acc: 0.8544 -- iter: 11392/55154
--
Training Step: 2676  | total loss: 0.06846 | time: 3420.671s
| Adam | epoch: 007 | loss: 0.06846 - acc: 0.9474 | val_loss: 0.49960 - val_acc: 0.8483 -- iter: 11520/55154
--
Training Step: 2677  | total loss: 0.06403 | time: 3457.267s
| Adam | epoch: 007 | loss: 0.06403 - acc: 0.9496 | val_loss: 0.51099 - val_acc: 0.8455 -- iter: 11648/55154
--
Training Step: 2678  | total loss: 0.06594 | time: 3494.248s
| Adam | epoch: 007 | loss: 0.06594 - acc: 0.9492 | val_loss: 0.49827 - val_acc: 0.8476 -- iter: 11776/55154
--
Training Step: 2679  | total loss: 0.06553 | time: 3533.640s
| Adam | epoch: 007 | loss: 0.06553 - acc: 0.9449 | val_loss: 0.48067 - va

Training Step: 2718  | total loss: 0.08100 | time: 5067.666s
| Adam | epoch: 007 | loss: 0.08100 - acc: 0.9548 | val_loss: 0.61906 - val_acc: 0.8297 -- iter: 16896/55154
--
Training Step: 2719  | total loss: 0.08101 | time: 5105.846s
| Adam | epoch: 007 | loss: 0.08101 - acc: 0.9531 | val_loss: 0.56778 - val_acc: 0.8382 -- iter: 17024/55154
--
Training Step: 2720  | total loss: 0.08118 | time: 5146.909s
| Adam | epoch: 007 | loss: 0.08118 - acc: 0.9539 | val_loss: 0.54581 - val_acc: 0.8415 -- iter: 17152/55154
--
Training Step: 2721  | total loss: 0.08158 | time: 5184.758s
| Adam | epoch: 007 | loss: 0.08158 - acc: 0.9546 | val_loss: 0.54911 - val_acc: 0.8406 -- iter: 17280/55154
--
Training Step: 2722  | total loss: 0.08148 | time: 5225.351s
| Adam | epoch: 007 | loss: 0.08148 - acc: 0.9552 | val_loss: 0.53582 - val_acc: 0.8427 -- iter: 17408/55154
--
Training Step: 2723  | total loss: 0.07924 | time: 5270.224s
| Adam | epoch: 007 | loss: 0.07924 - acc: 0.9558 | val_loss: 0.51211 - va

Training Step: 2762  | total loss: 0.08963 | time: 6760.739s
| Adam | epoch: 007 | loss: 0.08963 - acc: 0.9474 | val_loss: 0.52718 - val_acc: 0.8435 -- iter: 22528/55154
--
Training Step: 2763  | total loss: 0.09017 | time: 6798.291s
| Adam | epoch: 007 | loss: 0.09017 - acc: 0.9472 | val_loss: 0.56442 - val_acc: 0.8373 -- iter: 22656/55154
--
Training Step: 2764  | total loss: 0.08732 | time: 6837.083s
| Adam | epoch: 007 | loss: 0.08732 - acc: 0.9494 | val_loss: 0.53921 - val_acc: 0.8410 -- iter: 22784/55154
--
Training Step: 2765  | total loss: 0.09564 | time: 6875.181s
| Adam | epoch: 007 | loss: 0.09564 - acc: 0.9474 | val_loss: 0.52328 - val_acc: 0.8426 -- iter: 22912/55154
--
Training Step: 2766  | total loss: 0.09649 | time: 6912.969s
| Adam | epoch: 007 | loss: 0.09649 - acc: 0.9480 | val_loss: 0.50281 - val_acc: 0.8443 -- iter: 23040/55154
--
Training Step: 2767  | total loss: 0.09543 | time: 6950.344s
| Adam | epoch: 007 | loss: 0.09543 - acc: 0.9485 | val_loss: 0.50556 - va

Training Step: 2806  | total loss: 0.08499 | time: 8504.105s
| Adam | epoch: 007 | loss: 0.08499 - acc: 0.9536 | val_loss: 0.51695 - val_acc: 0.8493 -- iter: 28160/55154
--
Training Step: 2807  | total loss: 0.08538 | time: 8540.867s
| Adam | epoch: 007 | loss: 0.08538 - acc: 0.9536 | val_loss: 0.49511 - val_acc: 0.8524 -- iter: 28288/55154
--
Training Step: 2808  | total loss: 0.08825 | time: 8579.087s
| Adam | epoch: 007 | loss: 0.08825 - acc: 0.9535 | val_loss: 0.50633 - val_acc: 0.8483 -- iter: 28416/55154
--
Training Step: 2809  | total loss: 0.08674 | time: 8616.190s
| Adam | epoch: 007 | loss: 0.08674 - acc: 0.9543 | val_loss: 0.51973 - val_acc: 0.8443 -- iter: 28544/55154
--
Training Step: 2810  | total loss: 0.08518 | time: 8654.570s
| Adam | epoch: 007 | loss: 0.08518 - acc: 0.9541 | val_loss: 0.53633 - val_acc: 0.8404 -- iter: 28672/55154
--
Training Step: 2811  | total loss: 0.08452 | time: 8696.867s
| Adam | epoch: 007 | loss: 0.08452 - acc: 0.9548 | val_loss: 0.55006 - va

Training Step: 2850  | total loss: 0.08868 | time: 10272.070s
| Adam | epoch: 007 | loss: 0.08868 - acc: 0.9502 | val_loss: 0.57551 - val_acc: 0.8373 -- iter: 33792/55154
--
Training Step: 2851  | total loss: 0.08688 | time: 10309.852s
| Adam | epoch: 007 | loss: 0.08688 - acc: 0.9497 | val_loss: 0.59376 - val_acc: 0.8338 -- iter: 33920/55154
--
Training Step: 2852  | total loss: 0.08279 | time: 10348.458s
| Adam | epoch: 007 | loss: 0.08279 - acc: 0.9508 | val_loss: 0.63337 - val_acc: 0.8254 -- iter: 34048/55154
--
Training Step: 2853  | total loss: 0.08166 | time: 10387.849s
| Adam | epoch: 007 | loss: 0.08166 - acc: 0.9526 | val_loss: 0.63764 - val_acc: 0.8241 -- iter: 34176/55154
--
Training Step: 2854  | total loss: 0.09377 | time: 10427.536s
| Adam | epoch: 007 | loss: 0.09377 - acc: 0.9511 | val_loss: 0.61150 - val_acc: 0.8290 -- iter: 34304/55154
--
Training Step: 2855  | total loss: 0.09615 | time: 10468.060s
| Adam | epoch: 007 | loss: 0.09615 - acc: 0.9505 | val_loss: 0.5898

Training Step: 2893  | total loss: 0.08144 | time: 11931.408s
| Adam | epoch: 007 | loss: 0.08144 - acc: 0.9563 | val_loss: 0.60072 - val_acc: 0.8346 -- iter: 39296/55154
--
Training Step: 2894  | total loss: 0.08161 | time: 11968.474s
| Adam | epoch: 007 | loss: 0.08161 - acc: 0.9552 | val_loss: 0.57047 - val_acc: 0.8399 -- iter: 39424/55154
--
Training Step: 2895  | total loss: 0.08786 | time: 12005.650s
| Adam | epoch: 007 | loss: 0.08786 - acc: 0.9550 | val_loss: 0.54154 - val_acc: 0.8438 -- iter: 39552/55154
--
Training Step: 2896  | total loss: 0.09232 | time: 12042.429s
| Adam | epoch: 007 | loss: 0.09232 - acc: 0.9556 | val_loss: 0.54135 - val_acc: 0.8437 -- iter: 39680/55154
--
Training Step: 2897  | total loss: 0.09272 | time: 12080.528s
| Adam | epoch: 007 | loss: 0.09272 - acc: 0.9584 | val_loss: 0.55852 - val_acc: 0.8404 -- iter: 39808/55154
--
Training Step: 2898  | total loss: 0.09003 | time: 12120.074s
| Adam | epoch: 007 | loss: 0.09003 - acc: 0.9587 | val_loss: 0.5560

Training Step: 2936  | total loss: 0.07015 | time: 13666.674s
| Adam | epoch: 007 | loss: 0.07015 - acc: 0.9615 | val_loss: 0.51969 - val_acc: 0.8461 -- iter: 44800/55154
--
Training Step: 2937  | total loss: 0.06593 | time: 13706.578s
| Adam | epoch: 007 | loss: 0.06593 - acc: 0.9623 | val_loss: 0.55532 - val_acc: 0.8382 -- iter: 44928/55154
--
Training Step: 2938  | total loss: 0.06032 | time: 13748.989s
| Adam | epoch: 007 | loss: 0.06032 - acc: 0.9614 | val_loss: 0.58943 - val_acc: 0.8316 -- iter: 45056/55154
--
Training Step: 2939  | total loss: 0.06031 | time: 13786.936s
| Adam | epoch: 007 | loss: 0.06031 - acc: 0.9605 | val_loss: 0.59873 - val_acc: 0.8306 -- iter: 45184/55154
--
Training Step: 2940  | total loss: 0.06504 | time: 13825.125s
| Adam | epoch: 007 | loss: 0.06504 - acc: 0.9606 | val_loss: 0.55703 - val_acc: 0.8379 -- iter: 45312/55154
--
Training Step: 2941  | total loss: 0.07530 | time: 13863.125s
| Adam | epoch: 007 | loss: 0.07530 - acc: 0.9590 | val_loss: 0.5089

Training Step: 2979  | total loss: 0.09083 | time: 15337.900s
| Adam | epoch: 007 | loss: 0.09083 - acc: 0.9494 | val_loss: 0.55296 - val_acc: 0.8378 -- iter: 50304/55154
--
Training Step: 2980  | total loss: 0.09516 | time: 15375.190s
| Adam | epoch: 007 | loss: 0.09516 - acc: 0.9497 | val_loss: 0.54881 - val_acc: 0.8387 -- iter: 50432/55154
--
Training Step: 2981  | total loss: 0.09485 | time: 15412.760s
| Adam | epoch: 007 | loss: 0.09485 - acc: 0.9485 | val_loss: 0.52036 - val_acc: 0.8471 -- iter: 50560/55154
--
Training Step: 2982  | total loss: 0.09626 | time: 15450.363s
| Adam | epoch: 007 | loss: 0.09626 - acc: 0.9505 | val_loss: 0.52283 - val_acc: 0.8470 -- iter: 50688/55154
--
Training Step: 2983  | total loss: 0.09437 | time: 15487.923s
| Adam | epoch: 007 | loss: 0.09437 - acc: 0.9524 | val_loss: 0.52387 - val_acc: 0.8462 -- iter: 50816/55154
--
Training Step: 2984  | total loss: 0.09611 | time: 15525.120s
| Adam | epoch: 007 | loss: 0.09611 - acc: 0.9548 | val_loss: 0.5211

In [53]:
model.save(MODEL_NAME)

INFO:tensorflow:C:\Users\hiremath\Downloads\devanagari-0.001-5conv-basic.model is not in all_model_checkpoint_paths. Manually adding it.


In [59]:
TEST_DIR = 'C:\\Users\\hiremath\\Downloads\\devanagari\\Images\\test1'

In [60]:
def create_test_data():
    test_data = []
    for img in os.listdir(TEST_DIR):
        if img != "Thumbs - Copy.db" and img != "Thumbs.db":
            path1 = os.path.join(TEST_DIR,img)
            with Image.open(path1).convert('L') as img1:
                img1 = img1.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS)
                test_data.append([np.array(img1)])
    
    return test_data

In [61]:
test = create_test_data()

In [62]:
X = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)

In [63]:
v = len(X)

In [64]:
for i in range(0,v):
    scores = model.predict(X)[i]
    val = np.argmax(scores)
    print(val)

36
